# Predicción por Lotes con Intervalos de Confianza

En esta notebook, utilizaremos nuestro modelo XGBoost entrenado para realizar predicciones sobre un conjunto de datos de prueba que no ha visto. Los datos de prueba están particionados por código postal en formato Parquet.

El proceso será el siguiente:
1. Cargar el modelo entrenado y la lista de características.
2. Calcular un intervalo de confianza para las predicciones basándonos en los residuos del conjunto de prueba original.
3. Iterar sobre cada partición de datos (por código postal).
4. Realizar predicciones para cada lote.
5. Añadir la predicción y el intervalo de confianza al DataFrame.
6. Guardar los resultados en un nuevo directorio, manteniendo la estructura de particiones.

In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import joblib
import json
import os
from sklearn.model_selection import train_test_split

In [2]:
# Definir rutas
model_path = '../../models/xgboost_tuned_pipeline.joblib'
features_path = '../../data/processed/final_features.json'
original_data_path = '../../data/processed/final_processed_data.parquet'
test_data_dir = '../../data/processed/prod_por_codigo_postal/'
output_dir = '../../data/predictions/'

# Cargar modelo y características
model = joblib.load(model_path)
with open(features_path, 'r') as f:
    final_features = json.load(f)

print("Modelo y características cargados.")

Modelo y características cargados.


## 2. Calcular Límites para el Intervalo de Confianza usando Error Relativo Porcentual

Para que el intervalo de confianza sea más intuitivo y se adapte a la escala del precio, usaremos una variación porcentual en lugar de un monto absoluto. Un error de $10,000 es significativo para una propiedad barata, pero menor para una cara. El error porcentual captura esto.

El proceso es el siguiente:
1. Usar un conjunto de validación para el cual conocemos el precio real.
2. Realizar predicciones sobre este conjunto.
3. Calcular el **error relativo**: `(precio_real - predicción) / predicción`.
4. Crear contenedores de precios basados en las predicciones y, para cada uno, calcular los percentiles 5 y 95 del error relativo. Estos serán nuestros factores de ajuste (ej. -0.15 y +0.10).
5. Al predecir en nuevos datos, aplicaremos estos factores para definir el intervalo: `limite = prediccion * (1 + factor_error)`.

In [3]:
# Cargar los datos originales para crear un set de validación
original_df = pd.read_parquet(original_data_path)

# Dividir los datos para obtener un conjunto de validación (estratificado si es posible)
# Usamos el mismo random_state para asegurar la reproducibilidad
_, df_val = train_test_split(original_df, test_size=0.2, random_state=42)

# --- 1. Caracterización del Error Relativo en el Set de Validación ---

# Realizar predicciones en el set de validación
val_predictions = model.predict(df_val[final_features])

# Calcular error relativo: (real - pred) / pred
# Se añade un valor pequeño (epsilon) para evitar división por cero
epsilon = 1e-6
relative_errors = (df_val['precio_mxn'] - val_predictions) / (val_predictions + epsilon)

# Crear contenedores basados en las predicciones
prediction_bins = pd.qcut(val_predictions, q=10, labels=False, duplicates='drop')

# Calcular límites de error relativo por contenedor
error_factors = {}
for bin_label in np.unique(prediction_bins):
    bin_rel_errors = relative_errors[prediction_bins == bin_label]
    lower_factor = bin_rel_errors.quantile(0.05)
    upper_factor = bin_rel_errors.quantile(0.95)
    error_factors[bin_label] = (lower_factor, upper_factor)

# Necesitamos los umbrales de los contenedores para clasificar nuevas predicciones
_, bin_thresholds = pd.qcut(val_predictions, q=10, retbins=True, duplicates='drop')
bin_thresholds[0] = -np.inf # El primer límite es -infinito
bin_thresholds[-1] = np.inf # El último límite es +infinito

print("Factores de error relativo calculados a partir del set de validación.")

Factores de error relativo calculados a partir del set de validación.


In [4]:
error_factors

{0: (-0.5212149206533959, 0.7888172347117516),
 1: (-0.4694246513319183, 0.3872370661461857),
 2: (-0.3954911148858698, 0.4728319576045606),
 3: (-0.36359399879314225, 0.3557330582987902),
 4: (-0.36880268250821374, 0.4536850272730025),
 5: (-0.3209423313164633, 0.4010791935617052),
 6: (-0.3902653771326588, 0.4635978000503158),
 7: (-0.37005361345313287, 0.32664181599986475),
 8: (-0.42097986999172965, 0.35635344015893855),
 9: (-0.4628565347920143, 0.41068234913731505)}

In [5]:

# --- 2. Inferencia por Lotes aplicando los límites de error ---

os.makedirs(output_dir, exist_ok=True)

# Iterar sobre cada archivo de testeo particionado
for filename in os.listdir(test_data_dir):
    if filename.endswith('.parquet'):
        partition_path = os.path.join(test_data_dir, filename)
        df_partition = pd.read_parquet(partition_path)
        
        if df_partition.empty:
            continue
        
        #df_partition['es_penthouse'] = 0
        #rename_map = dict(zip(new_final_features, final_features))
        # Aplica el renombramiento al DataFrame
        #df_partition.rename(columns=rename_map, inplace=True)
        #df_partition[final_features] = df_partition[final_features].replace('*',0)
        #df_partition[final_features] = df_partition[final_features].astype(float)

        # Realizar predicciones sobre el lote
        predictions = model.predict(df_partition[final_features])
        # Asegurar que las predicciones no sean negativas
        predictions[predictions < 0] = None
        df_partition['prediction'] = predictions
        
        # Asignar cada predicción a un bin usando los umbrales calculados
        df_partition['prediction_bin'] = pd.cut(df_partition['prediction'], bins=bin_thresholds, labels=False, right=False)
        
        # Aplicar factores de error relativo
        df_partition['lower_bound'] = df_partition.apply(lambda row: row['prediction'] * (1 + error_factors.get(row['prediction_bin'], (0,0))[0]), axis=1)
        df_partition['upper_bound'] = df_partition.apply(lambda row: row['prediction'] * (1 + error_factors.get(row['prediction_bin'], (0,0))[1]), axis=1)
        
        # Asegurar que los límites del intervalo no sean negativos
        df_partition['lower_bound'] = df_partition['lower_bound'].clip(lower=0)
        df_partition['upper_bound'] = df_partition['upper_bound'].clip(lower=0)
        
        # Guardar resultados
        output_path = os.path.join(output_dir, filename)
        df_partition.to_parquet(output_path)

print("\nPredicciones por lotes con intervalos de confianza generadas.")


Predicciones por lotes con intervalos de confianza generadas.
